## Dataset

In [28]:
import cv2
import os
import glob
from sklearn.utils import shuffle
import numpy as np


In [29]:
def load_train(train_path, image_size, classes):
    images = []
    labels = []
    img_names = []
    cls = []

    print('Going to read')
    for fields in classes:   
        index = classes.index(fields)
        print('Going to read {} files (Index: {})'.format(fields, index))
        path = os.path.join(train_path, fields, '*g')
        files = glob.glob(path)
        for fl in files:
            image = cv2.imread(fl)
            print(fl)
            image = cv2.resize(image, (image_size, image_size),0,0, cv2.INTER_LINEAR)
            image = image.astype(np.float32)
            image = np.multiply(image, 1.0 / 255.0)
            images.append(image)
            label = np.zeros(len(classes))
            label[index] = 1.0
            labels.append(label)
            flbase = os.path.basename(fl)
            img_names.append(flbase)
            cls.append(fields)
    images = np.array(images)
    labels = np.array(labels)
    img_names = np.array(img_names)
    cls = np.array(cls)

    return images, labels, img_names, cls


In [30]:
class DataSet(object):

  def __init__(self, images, labels, img_names, cls):
    self._num_examples = images.shape[0]

    self._images = images
    self._labels = labels
    self._img_names = img_names
    self._cls = cls
    self._epochs_done = 0
    self._index_in_epoch = 0

  @property
  def images(self):
    return self._images

  @property
  def labels(self):
    return self._labels

  @property
  def img_names(self):
    return self._img_names

  @property
  def cls(self):
    return self._cls

  @property
  def num_examples(self):
    return self._num_examples

  @property
  def epochs_done(self):
    return self._epochs_done

  def next_batch(self, batch_size):

    start = self._index_in_epoch
    self._index_in_epoch += batch_size

    if self._index_in_epoch > self._num_examples:

      self._epochs_done += 1
      start = 0
      self._index_in_epoch = batch_size
      assert batch_size <= self._num_examples
    end = self._index_in_epoch

    return self._images[start:end], self._labels[start:end], self._img_names[start:end], self._cls[start:end]


def read_train_sets(train_path, image_size, classes, validation_size):
  class DataSets(object):
    pass
  data_sets = DataSets()

  images, labels, img_names, cls = load_train(train_path, image_size, classes)
  images, labels, img_names, cls = shuffle(images, labels, img_names, cls)  

  if isinstance(validation_size, float):
    validation_size = int(validation_size * images.shape[0])

  validation_images = images[:validation_size]
  validation_labels = labels[:validation_size]
  validation_img_names = img_names[:validation_size]
  validation_cls = cls[:validation_size]

  train_images = images[validation_size:]
  train_labels = labels[validation_size:]
  train_img_names = img_names[validation_size:]
  train_cls = cls[validation_size:]

  data_sets.train = DataSet(train_images, train_labels, train_img_names, train_cls)
  data_sets.valid = DataSet(validation_images, validation_labels, validation_img_names, validation_cls)

  return data_sets


## Training

In [31]:
#import dataset
import tensorflow as tf
import time
from datetime import timedelta
import math
import random
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib import colors

from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)


In [32]:
batch_size = 32


classes = ['c1','c2','c3','c4','c5','c6','c7','c8','c9','c10','d1','d2','d3','d4','d6','d9','d12','d14','d15','d16','o1','o2','o3','o4','o5','o6','o7','o8','o10','o11','o12','o13','o14','o15','o16','o17','o18','o19']
class_number = len(classes)

train_accuracy=[]
validation_accuracy=[]
validation_loss=[]
epoc_list=[]

validation_size = 0.2
image_size = 128
channel_number = 3
train_path='train_data'


data = read_train_sets(train_path, image_size, classes, validation_size=validation_size)


print("successfully read")
print("Training-set:\t\t{}".format(len(data.train.labels)))
print("Validation-set:\t{}".format(len(data.valid.labels)))



session = tf.Session()
x = tf.placeholder(tf.float32, shape=[None, image_size,image_size,channel_number], name='x')


y_true = tf.placeholder(tf.float32, shape=[None, class_number], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)


filter_size_con1 = 3
filter_number_con1 = 32

filter_size_con2 = 3
filter_number_con2 = 32

filter_size_con3 = 3
filter_number_con3 = 64

filter_size_con4 = 3
filter_number_con4 = 64

fc_layer_size = 128

Going to read
Going to read c1 files (Index: 0)
train_data/c1/001_2.png
train_data/c1/001_22.png
train_data/c1/001_19.png
train_data/c1/001_9.png
train_data/c1/001_17.png
train_data/c1/001_5.png
train_data/c1/001_15.png
train_data/c1/001_11.png
train_data/c1/001_14.png
train_data/c1/001_18.png
train_data/c1/001_3.png
train_data/c1/001_4.png
train_data/c1/001_1.png
train_data/c1/001_13.png
train_data/c1/001_7.png
train_data/c1/001_12.png
train_data/c1/001_6.png
train_data/c1/001_20.png
train_data/c1/001_10.png
train_data/c1/001_8.png
train_data/c1/001_21.png
train_data/c1/001_16.png
Going to read c2 files (Index: 1)
train_data/c2/002_5.png
train_data/c2/002_16.png
train_data/c2/002_21.png
train_data/c2/002_11.png
train_data/c2/002_22.png
train_data/c2/002_4.png
train_data/c2/002_8.png
train_data/c2/002_1.png
train_data/c2/002_6.png
train_data/c2/002_7.png
train_data/c2/002_10.png
train_data/c2/002_18.png
train_data/c2/002_20.png
train_data/c2/002_12.png
train_data/c2/002_13.png
train_da

train_data/o5/5_2.png
train_data/o5/5_21.png
train_data/o5/5_20.png
train_data/o5/5_11.png
train_data/o5/5_3.png
train_data/o5/5_18.png
train_data/o5/5_9.png
Going to read o6 files (Index: 25)
train_data/o6/6_1.png
train_data/o6/6_6.png
train_data/o6/6_12.png
train_data/o6/6_8.png
train_data/o6/6_13.png
train_data/o6/6_20.png
train_data/o6/6_11.png
train_data/o6/6_4.png
train_data/o6/6_14.png
train_data/o6/6_15.png
train_data/o6/6_5.png
train_data/o6/6_22.png
train_data/o6/6_9.png
train_data/o6/6_17.png
train_data/o6/6_2.png
train_data/o6/6_16.png
train_data/o6/6_21.png
train_data/o6/6_3.png
train_data/o6/6_7.png
train_data/o6/6_10.png
train_data/o6/6_18.png
train_data/o6/6_19.png
Going to read o7 files (Index: 26)
train_data/o7/7_12.png
train_data/o7/7_3.png
train_data/o7/7_7.png
train_data/o7/7_19.png
train_data/o7/7_4.png
train_data/o7/7_11.png
train_data/o7/7_16.png
train_data/o7/7_22.png
train_data/o7/7_17.png
train_data/o7/7_5.png
train_data/o7/7_13.png
train_data/o7/7_10.png
tra

In [33]:
def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def create_biases(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))


def create_convolutional_layer(input,
               input_channel_number, 
               conv_filter_size,        
               filter_number):  
    

    weights = create_weights(shape=[conv_filter_size, conv_filter_size, input_channel_number, filter_number])
    biases = create_biases(filter_number)


    layer = tf.nn.conv2d(input=input,
                     filter=weights,
                     strides=[1, 1, 1, 1],
                     padding='SAME')

    layer += biases


    layer = tf.nn.max_pool(value=layer,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')

    layer = tf.nn.relu(layer)

    return layer


def create_flatten_layer(layer):
    layer_shape = layer.get_shape()
    feature_number = layer_shape[1:4].num_elements()
    layer = tf.reshape(layer, [-1, feature_number])

    return layer


def create_fc_layer(input,          
             input_number,    
             output_number,
             use_relu=True):
    

    weights = create_weights(shape=[input_number, output_number])
    biases = create_biases(output_number)

    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer


### Convolution Layer

In [34]:
layer_con1 = create_convolutional_layer(input=x,
               input_channel_number=channel_number,
               conv_filter_size=filter_size_con1,
               filter_number=filter_number_con1)
layer_con2 = create_convolutional_layer(input=layer_con1,
               input_channel_number=filter_number_con1,
               conv_filter_size=filter_size_con2,
               filter_number=filter_number_con2)

layer_con3= create_convolutional_layer(input=layer_con2,
               input_channel_number=filter_number_con2,
               conv_filter_size=filter_size_con3,
               filter_number=filter_number_con3)

layer_con4= create_convolutional_layer(input=layer_con3,
               input_channel_number=filter_number_con3,
               conv_filter_size=filter_size_con4,
               filter_number=filter_number_con4)
          
flat_layer = create_flatten_layer(layer_con4)

layer_fc1 = create_fc_layer(input=flat_layer,
                     input_number=flat_layer.get_shape()[1:4].num_elements(),
                     output_number=fc_layer_size,
                     use_relu=True)

layer_fc2 = create_fc_layer(input=layer_fc1,
                     input_number=fc_layer_size,
                     output_number=fc_layer_size,
                     use_relu=True) 

					 

layer_fc3 = create_fc_layer(input=layer_fc2,
                     input_number=fc_layer_size,
                     output_number=class_number,
                     use_relu=False) 

In [35]:
y_pred = tf.nn.softmax(layer_fc3,name='y_pred')

y_pred_cls = tf.argmax(y_pred, dimension=1)
session.run(tf.global_variables_initializer())
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc3,
                                                    labels=y_true)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


session.run(tf.global_variables_initializer()) 

In [48]:
def show_progress(epoch, feed_dict_train, feed_dict_validate, val_loss):
	
	acc = session.run(accuracy, feed_dict=feed_dict_train)
	val_acc = session.run(accuracy, feed_dict=feed_dict_validate)
	msg = "Training Epoch {0} --- Training Accuracy: {1:>6.1%}, Validation Accuracy: {2:>6.1%},  Validation Loss: {3:.3f}"
	print (msg.format (epoch + 1, acc, val_acc, val_loss) )

total_iterations = 0

saver = tf.train.Saver()
def train(iteration_number):
    global total_iterations
    print ('Hello: ' +os.getcwd())
    for i in range(total_iterations,
                   total_iterations + iteration_number):

        x_batch, y_true_batch, _, cls_batch = data.train.next_batch(batch_size)
        x_valid_batch, y_valid_batch, _, valid_cls_batch = data.valid.next_batch(batch_size)

        
        feed_dict_tr = {x: x_batch,
                           y_true: y_true_batch}
        feed_dict_val = {x: x_valid_batch,
                              y_true: y_valid_batch}

        session.run(optimizer, feed_dict=feed_dict_tr)

        if i % int(data.train.num_examples/batch_size) == 0: 
            val_loss = session.run(cost, feed_dict=feed_dict_val)
            epoch = int(i / int(data.train.num_examples/batch_size))    
            
            show_progress(epoch, feed_dict_tr, feed_dict_val, val_loss)
            saver.save(session, os.getcwd()+'\\signature')
    total_iterations += iteration_number
train(iteration_number=200)

Hello: /home/bishwa/Documents/ML/Signature Detection
Training Epoch 1 --- Training Accuracy: 100.0%, Validation Accuracy:  90.6%,  Validation Loss: 0.187
Training Epoch 2 --- Training Accuracy: 100.0%, Validation Accuracy:  90.6%,  Validation Loss: 0.186
Training Epoch 3 --- Training Accuracy: 100.0%, Validation Accuracy:  90.6%,  Validation Loss: 0.186
Training Epoch 4 --- Training Accuracy: 100.0%, Validation Accuracy:  90.6%,  Validation Loss: 0.186
Training Epoch 5 --- Training Accuracy: 100.0%, Validation Accuracy:  90.6%,  Validation Loss: 0.185
Training Epoch 6 --- Training Accuracy: 100.0%, Validation Accuracy:  90.6%,  Validation Loss: 0.185
Training Epoch 7 --- Training Accuracy: 100.0%, Validation Accuracy:  90.6%,  Validation Loss: 0.185
Training Epoch 8 --- Training Accuracy: 100.0%, Validation Accuracy:  90.6%,  Validation Loss: 0.184
Training Epoch 9 --- Training Accuracy: 100.0%, Validation Accuracy:  90.6%,  Validation Loss: 0.184
Training Epoch 10 --- Training Accurac

### Predict

In [50]:
import cv2
import os
import glob
from sklearn.utils import shuffle
import tensorflow as tf
import numpy as np
import os, glob, cv2
import sys, argparse

In [57]:
path_dir = '/home/bishwa/Documents/ML/Signature%20Detection'
image_path = 'test_data'
filename = path_dir + '/' + image_path + '/'
path = os.path.join('test_data', filename, '*g')
files = glob.glob(path)


classes = ['c1','c2','c3','c4','c5','c6','c7','c8','c9','c10','d1','d2','d3','d4','d6','d9','d12','d14','d15','d16','o1','o2','o3','o4','o5','o6','o7','o8','o10','o11','o12','o13','o14','o15','o16','o17','o18','o19']
class_number = len(classes)
image_size = 128
channel_number = 3

totalExperimented =0;
truePositive=0;

In [ ]:
for i in files:

    filename = i

    p = 0
    className = ""
    chk2=0;
    i = i[::-1]
    for kk in i:
        if kk == '.':
            chk2=1;
        elif kk == '\\':
            break;
        elif kk == '/':
            break;
        elif chk2 == 1:
            className = className + kk;


    className = className[::-1]

    images = []
    image = cv2.imread(filename)
    image = cv2.resize(image, (image_size, image_size), 0, 0, cv2.INTER_LINEAR)
    images.append(image)
    images = np.array(images, dtype=np.uint8)
    images = images.astype('float32')
    images = np.multiply(images, 1.0 / 255.0)
    x_batch = images.reshape(1, image_size, image_size, channel_number)

    sess = tf.Session()
    saver = tf.train.import_meta_graph('signature.meta')
    saver.restore(sess, tf.train.latest_checkpoint('./'))
    graph = tf.get_default_graph()

    y_pred = graph.get_tensor_by_name("y_pred:0")
    x = graph.get_tensor_by_name("x:0")
    y_true = graph.get_tensor_by_name("y_true:0")
    y_test_images = np.zeros((1, class_number))

    feed_dict_testing = {x: x_batch, y_true: y_test_images}
    result = sess.run(y_pred, feed_dict=feed_dict_testing)
 
    j = 1;
    val =0;
    probableClass = 0;
    for ii in result:
        chk=0
        for jj in ii:
            chk=chk+1
            if jj>val:
                val =jj
                probableClass = chk
    totalExperimented = totalExperimented +1
    if val >= 0.90:

        print('Photo Name       : ', className)
        print('predicted person : ', classes[probableClass - 1])
        print('Probability value: ', val)
        truePositive = truePositive + 1
    else:
        print('Photo Name       : ', className)
        print('predicted person : Unknown')
        print('Probability value: ', val)

    print()
    print()
